## Colab 환경에서 환경설정
- 앞으로의 남은 실습은 Colab에서 실행한다는 가정하에 실습을 진행합니다.

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# findspark를 사용해 Spark 환경 초기화
import findspark

# findspark는 Python 환경에서 PySpark를 사용할 수 있도록 Spark 경로를 자동으로 설정해주는 라이브러리입니다.
# findspark.init()은 환경 변수를 설정해 PySpark가 올바르게 작동하도록 합니다.
# init() 메서드에 경로를 지정하지 않으면, 기본적으로 SPARK_HOME 환경 변수를 자동으로 검색합니다.
findspark.init("/content/spark-3.2.4-bin-hadoop3.2")

- data 디렉토리에 있는 파일을 drive의 /content 경로에 올려주세요.

## 라이브러리 및 spark 시작 및 처리를 위한 객체 생성

In [35]:
# PySpark 라이브러리를 임포트합니다. PySpark는 대규모 데이터를 병렬로 처리하고 분석하는 데 사용됩니다.
import pyspark

# DataFrame 및 SQL 작업을 수행하기 위해 SparkSession을 임포트합니다.
# SparkSession은 PySpark에서 DataFrame 및 SQL 작업을 위한 진입점 역할을 합니다.
from pyspark.sql import SparkSession

# Spark 설정을 구성하고 SparkContext를 생성하기 위해 필요한 모듈을 임포트합니다.
# SparkConf는 Spark 애플리케이션의 설정을 정의하는 데 사용되고,
# SparkContext는 RDD API를 사용하여 데이터 처리를 수행하기 위한 기본 객체입니다.
from pyspark import SparkConf, SparkContext

# 새로운 SparkSession을 생성합니다. 'optim'은 애플리케이션 이름으로,
# Spark UI 및 로그에 표시됩니다. SparkSession은 DataFrame과 SQL 작업을 수행하는 데 사용됩니다.
spark = SparkSession.builder.appName('optim').getOrCreate()

# SparkContext 객체를 가져옵니다. SparkContext는 RDD API 작업을 수행할 때 사용되는 기본 객체입니다.
# RDD(Resilient Distributed Dataset)는 PySpark의 저수준 API로, 데이터를 분산 처리하기 위해 사용됩니다.
sc = spark.sparkContext

## 참고
- **즉시 실행**: 파이썬의 기본 리스트 연산이나 판다스(Pandas) 같은 데이터 처리 라이브러리는 코드가 실행될 때마다 즉시 수행됩니다.
- **지연 평가**: Spark의 RDD 연산은 지연 평가(lazy evaluation) 방식으로 수행됩니다. 변환 연산(transformations)인 `map`이나 `filter`는 즉시 실행되지 않고, `min`, `max`, `mean` 같은 액션이 호출될 때 한꺼번에 실행됩니다.
- 지연 평가는 실행 계획을 최적화하여 불필요한 데이터 처리를 피하고 작업을 효율적으로 수행할 수 있도록 도와줍니다.


## 1. 데이터 파싱 및 결측치 제거
- `parseLine` 함수: 각 행을 쉼표로 분할해 승객 ID, 생존 여부, 나이, 티켓 요금을 추출하며, 결측치는 None으로 처리합니다.
- RDD 생성: `sc.textFile`을 사용해 CSV 파일을 불러와 RDD를 만들고, 헤더 행을 제거해 본문 데이터만 남깁니다.
- 결측치 제거: `filter`를 통해 나이와 티켓 요금이 None이 아닌 행만 남겨 유효한 데이터를 확보합니다.
- 데이터 개수 확인: `count` 메서드로 정제된 데이터의 개수를 출력해 데이터 정제 결과를 확인합니다.

In [36]:
# 각 행을 파싱하는 함수 정의
def parseLine(line):
    fields = line.split(',')
    passengerID = fields[0]  # 승객 ID
    survived = int(fields[1])  # 생존 여부: 0(사망), 1(생존)
    
    # 나이와 티켓 요금을 처리할 때, 비어 있는 경우를 확인합니다.
    try:
        age = float(fields[6]) if fields[6] else None  # 비어 있으면 None으로 처리
        fare = float(fields[10]) if fields[10] else None  # 비어 있으면 None으로 처리
    except ValueError:
        age, fare = None, None  # 예외가 발생하면 None으로 처리

    return (passengerID, survived, age, fare)


In [ ]:
# CSV 파일을 RDD로 로드
data_rdd = sc.textFile("train.csv")

# 첫 번째 행(헤더) 제거
header = data_rdd.first()  # 첫 번째 행을 가져옵니다.
data_rdd = data_rdd.filter(lambda row: row != header)  # 헤더와 일치하지 않는 행만 남깁니다.

# 데이터 파싱 및 RDD 생성
parsedLines = data_rdd.map(parseLine)

# 결측치가 있는 행 제거 (age 또는 fare가 None인 경우 필터링)
parsedLines = parsedLines.filter(lambda x: x[2] is not None and x[3] is not None)

# 데이터 개수 출력
data_count = parsedLines.count()
print(f"결측치를 제거한 후의 데이터 개수: {data_count}")

# 결과 확인 (예시)
parsedLines.take(5)

결측치를 제거한 후의 데이터 개수: 714


[('1', 0, 22.0, 7.25),
 ('2', 1, 38.0, 71.2833),
 ('3', 1, 26.0, 7.925),
 ('4', 1, 35.0, 53.1),
 ('5', 0, 35.0, 8.05)]

## 2. 생존자와 사망자 필터링
- parsedLinse에 이전에 익혔던 filter 함수를 사용하여 survivors, nonSurvivors에 대해서 필터링을 합니다.

In [38]:
# 생존자와 사망자를 필터링
survivors = parsedLines.filter(lambda x: x[1] == 1)
nonSurvivors = parsedLines.filter(lambda x: x[1] == 0)

## 4. 생존자와 사망자를 이용해 최소, 최대, 평균 나이 계산
- survivors와 nonSurvivors RDD를 사용하여 생존자와 사망자의 나이를 각각 분석합니다. 
- survivors는 생존한 승객들로 구성된 RDD이고, nonSurvivors는 사망한 승객들로 구성된 RDD입니다.
- 이 두 RDD를 활용하여 각 그룹의 최소 나이, 최대 나이, 그리고 평균 나이를 계산합니다.

In [ ]:
# 생존자와 사망자 그룹의 최소 및 최대 나이를 계산
# TODO: survivors, nonSurvivors 변수를 이용하여 이전에 실습했던 내용을 기반으로 함수를 완성하세요.
# 각 변수의 prefix[:3]를 min, max, avg의 요구에 맞게 작성하세요.
minAgeSurvivors = _________________________________
maxAgeSurvivors = _________________________________
avgAgeSurvivors = _________________________________


minAgeNonSurvivors = _________________________________
maxAgeNonSurvivors = _________________________________
avgAgeNonSurvivors = _________________________________


## 3. 생존자와 사망자를 이용해 최소, 최대, 평균 티켓 요금 계산
- survivors와 nonSurvivors RDD를 사용하여 생존자와 사망자의 티켓 요금을 각각 분석합니다.
- survivors는 생존한 승객들로 구성된 RDD이고, nonSurvivors는 사망한 승객들로 구성된 RDD입니다.
- 이 두 RDD를 활용하여 각 그룹의 최소 티켓 요금, 최대 티켓 요금, 그리고 평균 티켓 요금을 계산합니다.


In [ ]:
# 생존자와 사망자 그룹의 최소 및 최대 티켓 요금을 계산
# TODO: survivors, nonSurvivors 변수를 이용하여 이전에 실습했던 내용을 기반으로 함수를 완성하세요.
# 각 변수의 prefix[:3]를 min, max, avg의 요구에 맞게 작성하세요.
minFareSurvivors = _________________________________
maxFareSurvivors = _________________________________
avgFareSurvivors = _________________________________

minFareNonSurvivors = _________________________________
maxFareNonSurvivors = _________________________________
avgFareNonSurvivors = _________________________________

In [42]:
# 결과 출력
print(f"생존자의 최소 나이: {minAgeSurvivors:.2f}세")
print(f"생존자의 최대 나이: {maxAgeSurvivors:.2f}세")
print(f"생존자의 평균 나이: {avgAgeSurvivors:.2f}세")

print(f"\n사망자의 최소 나이: {minAgeNonSurvivors:.2f}세")
print(f"사망자의 최대 나이: {maxAgeNonSurvivors:.2f}세")
print(f"사망자의 평균 나이: {avgAgeNonSurvivors:.2f}세")

print(f"\n생존자의 최소 티켓 요금: {minFareSurvivors:.2f}달러")
print(f"생존자의 최대 티켓 요금: {maxFareSurvivors:.2f}달러")
print(f"생존자의 평균 티켓 요금: {avgFareSurvivors:.2f}달러")

print(f"\n사망자의 최소 티켓 요금: {minFareNonSurvivors:.2f}달러")
print(f"사망자의 최대 티켓 요금: {maxFareNonSurvivors:.2f}달러")
print(f"사망자의 평균 티켓 요금: {avgFareNonSurvivors:.2f}달러")

# 결과
"""
생존자의 최소 나이: 0.42세
생존자의 최대 나이: 80.00세
생존자의 평균 나이: 28.34세

사망자의 최소 나이: 1.00세
사망자의 최대 나이: 74.00세
사망자의 평균 나이: 30.63세

생존자의 최소 티켓 요금: 0.00달러
생존자의 최대 티켓 요금: 512.33달러
생존자의 평균 티켓 요금: 51.84달러

사망자의 최소 티켓 요금: 0.00달러
사망자의 최대 티켓 요금: 263.00달러
사망자의 평균 티켓 요금: 22.97달러
"""

생존자의 최소 나이: 0.42세
생존자의 최대 나이: 80.00세
생존자의 평균 나이: 28.34세

사망자의 최소 나이: 1.00세
사망자의 최대 나이: 74.00세
사망자의 평균 나이: 30.63세

생존자의 최소 티켓 요금: 0.00달러
생존자의 최대 티켓 요금: 512.33달러
생존자의 평균 티켓 요금: 51.84달러

사망자의 최소 티켓 요금: 0.00달러
사망자의 최대 티켓 요금: 263.00달러
사망자의 평균 티켓 요금: 22.97달러
